# Evaluation with Azure AI Foundry

This notebook demonstrates how to evaluate data using custom evaluators and send the results to [Azure AI Foundry](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio).

### Prerequisites

- An Azure subscription.
- An Azure AI Foundry workspace.
- An Azure AI Foundry project.
- An Azure OpenAI resource.

### Install the required packages

```bash
pip install -r requirements.txt
```

### Create the following environment variables or add them to an `.env` file

```bash
AZURE_OPENAI_ENDPOINT=<your-azure-openai-endpoint>
AZURE_OPENAI_API_KEY=<your-azure-openai-api-key>
AZURE_OPENAI_DEPLOYMENT=<your-azure-openai-deployment>
AZURE_OPENAI_API_VERSION=<your-azure-openai-api-version>
AZURE_SUBSCRIPTION_ID=<your-azure-subscription-id>
AZURE_RESOURCE_GROUP=<your-azure-resource-group>
AZURE_AI_FOUNDRY_PROJECT=<your-azure-azure_foundry_project>
```

### References

- [Azure AI Foundry](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio)
- [Evaluate your Generative AI application locally with the Azure AI Evaluation SDK](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk#evaluating-direct-and-indirect-attack-jailbreak-vulnerability)

In [1]:
!pip install -r requirements.txt

## Imports

In [2]:
import json
import pandas as pd

In [3]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from promptflow.core import AzureOpenAIModelConfiguration
from promptflow.tracing import start_trace

if "AZURE_OPENAI_API_KEY" not in os.environ:
    # load environment variables from .env file
    load_dotenv()

# start a trace session, and print a url for user to check trace
start_trace()

## Setup Credentials and Configuration

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()


azure_ai_project = {
    "subscription_id": os.getenv("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.getenv("AZURE_RESOURCE_GROUP"),
    "project_name": os.getenv("AZURE_AI_FOUNDRY_PROJECT"),
}


model_config = {
    "api_key":os.getenv("AZURE_OPENAI_API_KEY"),
    "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "azure_deployment": os.getenv("AZURE_OPENAI_DEPLOYMENT"),
}



configuration = AzureOpenAIModelConfiguration(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version = os.getenv("AZURE_OPENAI_API_VERSION"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

credential = DefaultAzureCredential()

In [5]:
print(azure_ai_project)



{'subscription_id': '0debeb64-562c-44d8-9966-110450d5f9ed', 'resource_group_name': 'paramount-data-analytics', 'project_name': 'paramountda01'}


## Groundedness Evaluator

In [7]:
from azure.ai.evaluation import GroundednessProEvaluator, GroundednessEvaluator

# Initialazing Groundedness and Groundedness Pro evaluators
groundedness_eval = GroundednessEvaluator(model_config, threshold=3)

query_response_pass = dict(
    query="are partments for cargo freight exempt under the travel exemption?",
    context="",
    response="No. According to the exemption language set forth in several of these documents – for example, [31 CFR § 545.208] and [31 CFR § 536.205] – the travel exemption applies only to transactions \"ordinarily incident to travel.\" That phrase is defined to include items such as the export or importation of accompanied baggage for personal use, payment of living expenses, and arrangements or facilitation of travel. In contrast, arrangements or transactions related to cargo freight – including any compartments or facilities used solely for the transport of commercial cargo – are not covered by this exemption. In other words, because cargo freight shipments (or \"parts\" thereof) do not fall into the category of matters incident to personal travel, they do not benefit from the travel exemption provided under these regulations. None of the documents indicate that any commercial cargo freight or compartments used for it are treated as \"accompanied baggage\" or otherwise incident to travel for personal use. The commentary accompanying these provisions further reinforces that the travel exemption is intended solely for travel-related transactions (whether for accompanied baggage or for travel arrangements), not for the exportation of goods or commercial freight services."
)

query_response_fail = dict(
    query="are partments for cargo freight exempt under the travel exemption?",
    context="",
    response="Yes. According to [Case No. IA-16194], U.S. airlines generally may rely on the travel exemption to enter into interline ticketing and baggage (ITB) and special prorate agreements (SPAs) with Iranian airlines—provided the agreements are directly incident to arranging or facilitating international travel to or from Iran. [Case No. IA-16194] explains that while the Iranian Transactions Regulations impose broad prohibitions on transactions with Iranian goods or services, the exemption in § 560.210(d) covers transactions ordinarily incident to travel. In this context, interline ticketing and prorate agreements are considered part of travel arrangements, as long as they are confined to facilitating travel. The guidance in [Case No. IA-16194] also emphasizes that in order to fully determine exemption status, additional details must be provided about the specific nature of the ITB and SPA agreements—such as whether the agreements include any components (for example, ancillary goods or services) that may fall outside the direct facilitation of travel. Moreover, it is noted that the exemption does not apply to entities designated under the Global Terrorism Sanctions Regulations (GTSR) (see commentary in [Case No. IA-16194]), so if any counterparties are so designated, the exemption would not apply. In summary, for U.S. airlines: • If an interline ticketing and baggage or special prorate agreement is strictly limited to activities directly incident to international travel (e.g., issuing tickets, coordinating baggage transfer, or proration of travel fares), then such transactions are exempt under the travel exemption. • If the agreements include additional non–travel-related components or involve counterparties designated under certain sanctions (such as under the GTSR), then the exemption may not apply. This analysis is based on the information and guidance provided in [Case No. IA-16194]."
)

# Running Groundedness Evaluator on a query and response pair
groundedness_score = groundedness_eval(
    **query_response_fail
)
print(groundedness_score)



{'groundedness': 2.0, 'gpt_groundedness': 2.0, 'groundedness_reason': 'The response is related to the topic of travel exemptions but does not directly address the specific query about apartments for cargo freight. It provides detailed information about interline ticketing and prorate agreements, which is not relevant to the query.', 'groundedness_result': 'fail', 'groundedness_threshold': 3}


In [23]:
records = []
with open("data/data.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        result = groundedness_eval(
            query=item["query"],
            context=item.get("context", ""),
            response=item["ground_truth"]
        )
        records.append({
            "query": item["query"],
            "response": item["ground_truth"],
            "groundedness": result["groundedness"],
            "groundedness_reason": result["groundedness_reason"],
            "groundedness_result": result["groundedness_result"],
            "groundedness_threshold": result["groundedness_threshold"],
        })

# Load into a DataFrame and display
df = pd.DataFrame(records)
pd.set_option('display.max_colwidth', None)
display(df)


,query,response,groundedness,groundedness_reason,groundedness_result,groundedness_threshold
0,are partments for cargo freight exempt under the travel exemption?,"No. According to the exemption language set forth in several of these documents – for example, [31 CFR § 545.208] and [31 CFR § 536.205] – the travel exemption applies only to transactions ""ordinarily incident to travel."" That phrase is defined to include items such as the export or importation of accompanied baggage for personal use, payment of living expenses, and arrangements or facilitation of travel. In contrast, arrangements or transactions related to cargo freight – including any compartments or facilities used solely for the transport of commercial cargo – are not covered by this exemption.\nIn other words, because cargo freight shipments (or “parts” thereof) do not fall into the category of matters incident to personal travel, they do not benefit from the travel exemption provided under these regulations. None of the documents indicate that any commercial cargo freight or compartments used for it are treated as “accompanied baggage” or otherwise incident to travel for personal use.\nThe commentary accompanying these provisions further reinforces that the travel exemption is intended solely for travel-related transactions (whether for accompanied baggage or for travel arrangements), not for the exportation of goods or commercial freight services.",5.0,"The response is fully correct and complete, providing a detailed explanation based on specific regulations and directly addressing the query.",pass,3
1,are Interline Ticketing and Baggage and Special Prorate Agreements exempt under the travel exemption?,"Yes. According to [Case No. IA-16194], U.S. airlines generally may rely on the travel exemption to enter into interline ticketing and baggage (ITB) and special prorate agreements (SPAs) with Iranian airlines—provided the agreements are directly incident to arranging or facilitating international travel to or from Iran. [Case No. IA-16194] explains that while the Iranian Transactions Regulations impose broad prohibitions on transactions with Iranian goods or services, the exemption in § 560.210(d) covers transactions ordinarily incident to travel. In this context, interline ticketing and prorate agreements are considered part of travel arrangements, as long as they are confined to facilitating travel.\nThe guidance in [Case No. IA-16194] also emphasizes that to determine exemption status, additional details must be provided about the specific nature of the ITB and SPA agreements—such as whether the agreements include components that fall outside the direct facilitation of travel. Moreover, the exemption does not apply to entities designated under the Global Terrorism Sanctions Regulations (GTSR).\nSummary:\n• If the agreement is strictly limited to activities directly incident to travel (ticketing, baggage transfer, fare proration), it is exempt.\n• If it includes non-travel components or involves designated counterparties, it is not.\nThis analysis is based on [Case No. IA-16194].",5.0,"The RESPONSE is fully correct and complete, directly addressing the QUERY with precise information from the CONTEXT.",pass,3
2,"Is paying ""landings fees and ground handling payments"" to a blocked airport operator in connection with a charter flight exempt under the travel exemption?","No. According to [Case No. Ukraine-EO13662-2018-353695-1] and the related discussion in “Notable Examples Illustrating the Scope of the IEEPA Travel Exemption,” payments for landing fees and ground-handling services to a blocked airport operator are not treated as transactions “ordinarily incident to travel” and therefore are not covered by the travel exemption.\nThe case notes that although certain charter-flight operations may be authorized if they lack a U.S. nexus, OFAC did not indicate that the travel exemption applies to fees paid to blocked operators. The cited examples explicitly list su

## Relevance Evaluator

In [8]:
from azure.ai.evaluation import RelevanceEvaluator

# Initialazing Groundedness and Groundedness Pro evaluators
relevance_eval = RelevanceEvaluator(model_config, threshold=3)

query_response = dict(
    query="are partments for cargo freight exempt under the travel exemption?",
    context="",
    response="No. According to the exemption language set forth in several of these documents – for example, [31 CFR § 545.208] and [31 CFR § 536.205] – the travel exemption applies only to transactions \"ordinarily incident to travel.\" That phrase is defined to include items such as the export or importation of accompanied baggage for personal use, payment of living expenses, and arrangements or facilitation of travel. In contrast, arrangements or transactions related to cargo freight – including any compartments or facilities used solely for the transport of commercial cargo – are not covered by this exemption.In other words, because cargo freight shipments (or “parts” thereof) do not fall into the category of matters incident to personal travel, they do not benefit from the travel exemption provided under these regulations. None of the documents indicate that any commercial cargo freight or compartments used for it are treated as “accompanied baggage” or otherwise incident to travel for personal use.The commentary accompanying these provisions further reinforces that the travel exemption is intended solely for travel-related transactions (whether for accompanied baggage or for travel arrangements), not for the exportation of goods or commercial freight services.."
)

query_response_fail = dict(
    query="are partments for cargo freight exempt under the travel exemption?",
    context="",
    response="Yes. According to [Case No. IA-16194], U.S. airlines generally may rely on the travel exemption to enter into interline ticketing and baggage (ITB) and special prorate agreements (SPAs) with Iranian airlines—provided the agreements are directly incident to arranging or facilitating international travel to or from Iran. [Case No. IA-16194] explains that while the Iranian Transactions Regulations impose broad prohibitions on transactions with Iranian goods or services, the exemption in § 560.210(d) covers transactions ordinarily incident to travel. In this context, interline ticketing and prorate agreements are considered part of travel arrangements, as long as they are confined to facilitating travel. The guidance in [Case No. IA-16194] also emphasizes that in order to fully determine exemption status, additional details must be provided about the specific nature of the ITB and SPA agreements—such as whether the agreements include any components (for example, ancillary goods or services) that may fall outside the direct facilitation of travel. Moreover, it is noted that the exemption does not apply to entities designated under the Global Terrorism Sanctions Regulations (GTSR) (see commentary in [Case No. IA-16194]), so if any counterparties are so designated, the exemption would not apply. In summary, for U.S. airlines: • If an interline ticketing and baggage or special prorate agreement is strictly limited to activities directly incident to international travel (e.g., issuing tickets, coordinating baggage transfer, or proration of travel fares), then such transactions are exempt under the travel exemption. • If the agreements include additional non–travel-related components or involve counterparties designated under certain sanctions (such as under the GTSR), then the exemption may not apply. This analysis is based on the information and guidance provided in [Case No. IA-16194]."
)


# Running Groundedness Evaluator on a query and response pair
relevance_eval_score = relevance_eval(
    **query_response_pass
)
print(relevance_eval_score)

{'relevance': 4.0, 'gpt_relevance': 4.0, 'relevance_reason': 'The RESPONSE fully addresses the QUERY with accurate and complete information, including references to specific regulations and a detailed explanation of why cargo freight is not exempt under the travel exemption.', 'relevance_result': 'pass', 'relevance_threshold': 3}


In [22]:
records = []
with open("data/data.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        result = relevance_eval(
            query=item["query"],
            context=item.get("context", ""),
            response=item["ground_truth"]
        )
        records.append({
            "query":       item["query"],
            "response":    item["ground_truth"],
            "relevance":   result["relevance"],
            "relevant_reason": result["relevance_reason"],
            "relevant_result": result["relevance_result"],
            "relevant_threshold": result["relevance_threshold"],
         
        })

# 3. Load into a DataFrame and display
df = pd.DataFrame(records)
pd.set_option('display.max_colwidth', None)
display(df)

,query,response,relevance,relevant_reason,relevant_result,relevant_threshold
0,are partments for cargo freight exempt under the travel exemption?,"No. According to the exemption language set forth in several of these documents – for example, [31 CFR § 545.208] and [31 CFR § 536.205] – the travel exemption applies only to transactions ""ordinarily incident to travel."" That phrase is defined to include items such as the export or importation of accompanied baggage for personal use, payment of living expenses, and arrangements or facilitation of travel. In contrast, arrangements or transactions related to cargo freight – including any compartments or facilities used solely for the transport of commercial cargo – are not covered by this exemption.\nIn other words, because cargo freight shipments (or “parts” thereof) do not fall into the category of matters incident to personal travel, they do not benefit from the travel exemption provided under these regulations. None of the documents indicate that any commercial cargo freight or compartments used for it are treated as “accompanied baggage” or otherwise incident to travel for personal use.\nThe commentary accompanying these provisions further reinforces that the travel exemption is intended solely for travel-related transactions (whether for accompanied baggage or for travel arrangements), not for the exportation of goods or commercial freight services.",4.0,"The RESPONSE fully addresses the QUERY with accurate and complete information, explaining why cargo freight compartments are not exempt under the travel exemption. It includes all essential details required for a comprehensive understanding.",pass,3
1,are Interline Ticketing and Baggage and Special Prorate Agreements exempt under the travel exemption?,"Yes. According to [Case No. IA-16194], U.S. airlines generally may rely on the travel exemption to enter into interline ticketing and baggage (ITB) and special prorate agreements (SPAs) with Iranian airlines—provided the agreements are directly incident to arranging or facilitating international travel to or from Iran. [Case No. IA-16194] explains that while the Iranian Transactions Regulations impose broad prohibitions on transactions with Iranian goods or services, the exemption in § 560.210(d) covers transactions ordinarily incident to travel. In this context, interline ticketing and prorate agreements are considered part of travel arrangements, as long as they are confined to facilitating travel.\nThe guidance in [Case No. IA-16194] also emphasizes that to determine exemption status, additional details must be provided about the specific nature of the ITB and SPA agreements—such as whether the agreements include components that fall outside the direct facilitation of travel. Moreover, the exemption does not apply to entities designated under the Global Terrorism Sanctions Regulations (GTSR).\nSummary:\n• If the agreement is strictly limited to activities directly incident to travel (ticketing, baggage transfer, fare proration), it is exempt.\n• If it includes non-travel components or involves designated counterparties, it is not.\nThis analysis is based on [Case No. IA-16194].",5.0,"The RESPONSE fully addresses the QUERY with accurate and complete information, including relevant insights and conditions for the exemption.",pass,3
2,"Is paying ""landings fees and ground handling payments"" to a blocked airport operator in connection with a charter flight exempt under the travel exemption?","No. According to [Case No. Ukraine-EO13662-2018-353695-1] and the related discussion in “Notable Examples Illustrating the Scope of the IEEPA Travel Exemption,” payments for landing fees and ground-handling services to a blocked airport operator are not treated as transactions “ordinarily incident to travel” and therefore are not covered by the travel exemption.\nThe case notes that although certain charter-flight operations may be authorized if they lack a U.S. nexus, OFAC did not indica

## Friendliness Evaluator

In [14]:
from friendliness.friendliness import FriendlinessEvaluator

friendliness_eval = FriendlinessEvaluator(configuration)

friendliness_score = friendliness_eval(response="I will not apologize for my behavior!")

print(friendliness_score)

## Evaluate with both built-in and custom evaluators

In [16]:
import os
import pathlib
from pathlib import Path

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    ContentSafetyEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    FluencyEvaluator,
    SimilarityEvaluator,
)
from model_endpoint import ModelEndpoint

# Disable local snapshot to speed up evaluation
os.environ["PROMPTFLOW_DISABLE_LOCAL_SNAPSHOT"] = "true"

content_safety_evaluator = ContentSafetyEvaluator(
    azure_ai_project=azure_ai_project, credential=DefaultAzureCredential()
)
relevance_evaluator = RelevanceEvaluator(model_config)
coherence_evaluator = CoherenceEvaluator(model_config)
groundedness_evaluator = GroundednessEvaluator(model_config)
fluency_evaluator = FluencyEvaluator(model_config)
similarity_evaluator = SimilarityEvaluator(model_config)

# Create proper output path
output_path = str(Path.cwd() / "results.jsonl")
print(f"Output path: {output_path}")

# Pre-create the results file to avoid snapshot copy issues
Path("results.jsonl").touch()

results = evaluate(
    evaluation_name="Eval-Run-" + "-" + model_config["azure_deployment"].title(),
    data= "./data/data_3.jsonl",
    target=ModelEndpoint(model_config),
    evaluators={
        "content_safety": content_safety_evaluator,
        "coherence": coherence_evaluator,
        "relevance": relevance_evaluator,
        "groundedness": groundedness_evaluator,
        "fluency": fluency_evaluator,
        "similarity": similarity_evaluator,
        "friendliness": friendliness_eval #custom evaluator
    },
    # column mapping
    evaluator_config={
        "content_safety": {"column_mapping": {"query": "${data.query}", "response": "${data.response}"}},
        "coherence": {"column_mapping": {"response": "${data.response}", "query": "${data.query}"}},
        "relevance": {
            "column_mapping": {"response": "${data.response}", "context": "${data.context}", "query": "${data.query}"}
        },
        "groundedness": {
            "column_mapping": {
                "response": "${data.response}",
                "context": "${data.context}",
                "query": "${data.query}",
            }
        },
        "fluency": {
            "column_mapping": {"response": "${data.response}", "context": "${data.context}", "query": "${data.query}"}
        },
        "similarity": {
            "column_mapping": {"response": "${data.response}", "ground_truth": "${data.ground_truth}", "query": "${data.query}"}
        },
        "friendliness": {
            "column_mapping": {"response": "${data.response}", "context": "${data.context}", "query": "${data.query}"
            }
        }
    },
    # Optionally provide your Azure AI project information to track your evaluation results in your Azure AI project
    azure_ai_project = azure_ai_project,
    # Use proper output path
    output_path=output_path
)



[2025-06-17 11:51:58 -0400][promptflow._core.entry_meta_generator][WARNING] - Generate meta timeout after 60 seconds, terminate the process.
[2025-06-17 11:51:58 -0400][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_None_20250617_115055_193492, log path: C:\Users\akaruparti\.promptflow\.runs\azure_ai_evaluation_evaluators_None_20250617_115055_193492\logs.txt
[2025-06-17 12:00:03 -0400][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-06-17 12:00:03 -0400][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_groundedness_20250617_120002_885789, log path: C:\Users\akaruparti\.promptflow\.runs\azure_ai_evaluation_evaluators_groundedness_20250617_120002_885789\logs.txt
[2025-06-17 12:00:03 -0400][promptflow._core.entry_meta_generator][WARNING] - Generate meta in curren

2025-06-17 12:00:10 -0400   40844 execution.bulk     INFO     Finished 1 / 8 lines.
2025-06-17 12:00:10 -0400   40844 execution.bulk     INFO     Average execution time for completed lines: 6.83 seconds. Estimated time for incomplete lines: 47.81 seconds.
2025-06-17 12:00:10 -0400   40844 execution.bulk     INFO     Finished 2 / 8 lines.
2025-06-17 12:00:10 -0400   40844 execution.bulk     INFO     Average execution time for completed lines: 3.47 seconds. Estimated time for incomplete lines: 20.82 seconds.
2025-06-17 12:00:10 -0400   40844 execution.bulk     INFO     Finished 1 / 8 lines.
2025-06-17 12:00:10 -0400   40844 execution.bulk     INFO     Average execution time for completed lines: 5.93 seconds. Estimated time for incomplete lines: 41.51 seconds.
2025-06-17 12:00:11 -0400   40844 execution.bulk     INFO     Finished 3 / 8 lines.
2025-06-17 12:00:11 -0400   40844 execution.bulk     INFO     Average execution time for completed lines: 2.35 seconds. Estimated time for incomplet

In [17]:
import json
import pandas as pd
from IPython.display import display

# Load the JSON content from the file "results.jsonl"
with open("results.jsonl", "r") as f:
	jsonl_text = f.read()

data = json.loads(jsonl_text)

# Create a DataFrame using the "rows" key from the JSON data
df = pd.DataFrame(data.get("rows", []))

# Convert the DataFrame to CSV format as a string
csv_content = df.to_csv(index=False)
# Display the DataFrame as a table in the notebook
display(df)


,outputs.query,outputs.response,inputs.query,inputs.context,inputs.response,inputs.ground_truth,outputs.content_safety.violence,outputs.content_safety.violence_score,outputs.content_safety.violence_reason,outputs.content_safety.violence_threshold,...,outputs.fluency.fluency_reason,outputs.fluency.fluency_result,outputs.fluency.fluency_threshold,outputs.similarity.similarity,outputs.similarity.gpt_similarity,outputs.similarity.similarity_result,outputs.similarity.similarity_threshold,outputs.friendliness.score,outputs.friendliness.reason,line_number
0,are partments for cargo freight exempt under t...,The travel exemption rules can vary significan...,are partments for cargo freight exempt under t...,,No. According to the exemption language set fo...,No. According to the exemption language set fo...,Very low,0,The system's response provides a detailed expl...,3,...,"The response is well-articulated, with good co...",pass,3,5,5,pass,3,3,The response is neutral. It provides detailed ...,0
1,are Interline Ticketing and Baggage and Specia...,The details concerning exemptions under travel...,are Interline Ticketing and Baggage and Specia...,,"Yes. According to [Case No. IA-16194], U.S. ai...","Yes. According to [Case No. IA-16194], U.S. ai...",Very low,0,The system's response provides detailed inform...,3,...,"The response is well-articulated, with good co...",pass,3,5,5,pass,3,3,"The response is neutral, providing detailed in...",1
2,"Is paying ""landings fees and ground handling p...",The specifics of exemptions related to payment...,"Is paying ""landings fees and ground handling p...",,No. According to [Case No. Ukraine-EO13662-201...,No. According to [Case No. Ukraine-EO13662-201...,Very low,0,The system's response provides a detailed expl...,3,...,"The response is well-articulated, with good co...",pass,3,5,5,pass,3,3,"The response is neutral, providing factual inf...",2
3,are transactions related to Information housed...,"The ""informational materials exemption"" genera...",are transactions related to Information housed...,,"Yes. According to [Case No. IA-2012-293259-1],...","Yes. According to [Case No. IA-2012-293259-1],...",Very low,0,The system's response provides a legal explana...,3,...,The response demonstrates proficient fluency w...,pass,3,5,5,pass,3,3,"The response is neutral, providing factual inf...",3
4,would broadcasting an advertisement on behalf ...,Under the rules of the Office of Foreign Asset...,would broadcasting an advertisement on behalf ...,,"Yes. Broadcasting a pre-existing, “copy-ready”...","Yes. Broadcasting a pre-existing, “copy-ready”...",Very low,0,The system's response provides an informationa...,3,...,"The response is well-articulated, with good co...",pass,3,5,5,pass,3,3,The response is neutral. It provides informati...,4
5,are sculptures informational materials?,Sculptures are generally considered works of a...,are sculptures informational materials?,,"Yes. Under 31 CFR § 579.305, artworks classifi...","Yes. Under 31 CFR § 579.305, artworks classifi...",Very low,0,The system's response provides an informationa...,3,...,The response demonstrates proficient fluency w...,pass,3,5,5,pass,3,3,"The response is neutral, providing factual inf...",5
6,would it be prohibited to donate books to nort...,Donating books to North Korea can be a complex...,would it be prohibited to donate books to nort...,,Generally no. [Case No. DPRK2-2018-356417-1] s...,Generally no. [Case No. DPRK2-2018-356417-1] s...,Very low,0,The system's response provides an informationa...,3,...,"The response is well-articulated, with good co...",pass,3,5,5,pass,3,3,"The response is neutral, providing factual inf...",6
7,can iranian origin banknotes be imported into ...,"As of my last update in October 2023, the impo...",can iranian origin banknotes be imported into ...,,It depends. [Case No. IA-2018-355540-1] explai...,It depends. [Case No. IA-2018-355540-1] explai...,Very low,0,The system's response provides an informationa...,3,...,The

In [18]:
df.to_csv("results.csv", index=False)